In [229]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt
import phidl.geometry as pg
import shapely as sh
from phidl.quickplotter import quickplot2 as qp


def shapely_to_phidl(sh_pol):
    d1=pg.Device()
    d1.add_polygon(sh_pol.exterior.xy)
    
    d2=pg.Device()
    for interiors in sh_pol.interiors:
        d2.add_polygon(interiors.xy)
    return pg.boolean(d1,d2,operation='A-B')
def merge_line(line):
    new_linestring=[]
    for i in range(len(line)-1):
        l1=line[i]
        new_linestring.append(sh.LineString(l1))
        l2=line[i+1]
        l1_a,l1_b,l2_a,l2_b=l1[0,:],l1[-1,:],l2[0,:],l2[-1,:]
        connect_table=np.sum(np.array([l1_a-l2_a,l1_a-l2_b,l1_b-l2_a,l1_b-l2_b])**2,axis=1) #want l1_b-l2_a to be min
        if np.min(connect_table)!=0: #there's a gap between lines
            if np.argmin(connect_table)==0:
                new_linestring.append(sh.LineString([sh.Point(l1_a),sh.Point(l2_a)]))
            if np.argmin(connect_table)==1:
                new_linestring.append(sh.LineString([sh.Point(l1_a),sh.Point(l2_b)]))
            if np.argmin(connect_table)==2:
                new_linestring.append(sh.LineString([sh.Point(l1_b),sh.Point(l2_a)]))
            if np.argmin(connect_table)==3:
                new_linestring.append(sh.LineString([sh.Point(l1_b),sh.Point(l2_b)]))
    new_linestring.append(sh.LineString(line[-1]))
    return sh.line_merge(sh.unary_union(new_linestring))
def smooth_route(route):
    from geomdl import NURBS,knotvector
    # Create a 3-dimensional B-spline Curve
    curve = NURBS.Curve()

    # Set degree
    curve.degree = 3
    curve.ctrlpts = np.asarray(sh.LineString(route.interpolate(np.linspace(0,1,int(route.length/5)),normalized=True)).coords)
    curve.knotvector = knotvector.generate(curve.degree,len(curve.ctrlpts))
    # Set knot vector

    # Set evaluation delta (controls the number of curve points)
    curve.delta = 1/1000

    # Get curve points (the curve will be automatically evaluated)
    curve_points = np.asarray(curve.evalpts)[:,:]
    return sh.LineString(curve_points)



In [2]:
t = np.linspace(0,2*np.pi,100)
rad=50
sh_dev=[]
for ang in np.arange(0,2*np.pi,2/3*np.pi):
    cx,cy=np.cos(ang)*35,np.sin(ang)*35
    x,y=np.cos(t)*rad+cx,np.sin(t)*rad+cy
    sh_dev.append(sh.LineString( np.vstack((x,y)).T))
outer_circ = sh.LineString( np.vstack((200*np.cos(t),200*np.sin(t))).T).buffer(15)
three_rings=sh.unary_union(sh_dev).buffer(5)
cross_bar= (sh.LineString([sh.Point(-300,-300),sh.Point(300,300)])).buffer(8)
cutout=sh.Polygon(outer_circ.interiors[0]).difference(sh.Polygon(three_rings.exterior))
cross_bar=cross_bar.intersection(cutout)

dev = outer_circ.union(three_rings).union(cross_bar).buffer(5).buffer(-5)
qp(shapely_to_phidl(dev))

In [3]:
import pygmsh,gmsh
import numpy as np

def get_coordinates(line):
    return np.asarray(line.xy).T[::2,:][:-1,:]

with pygmsh.occ.Geometry() as geom:
    outline=geom.add_polygon( get_coordinates(dev.exterior))
    holes= [geom.add_polygon( get_coordinates(loop) ) for loop in dev.interiors]
    geom.boolean_difference(outline, holes)
   
    gmsh.option.setNumber('Mesh.MeshSizeMax',5)

    mesh =geom.generate_mesh()

In [4]:
x,y,_=mesh.points.T
tri=mesh.cells[1].data
plt.triplot(x,y,triangles=tri)

In [5]:
from centerline.geometry import Centerline
skeleton=sh.line_merge(Centerline(dev).geometry)

for el in skeleton.geoms:
    plt.plot(*el.xy)

In [6]:
import utilities as util

In [7]:
router=util.RoutingLayout(dev,border_tolerance=2)

In [8]:
plt.plot(*dev.exterior.xy,color='b')
for interior in dev.interiors:
    plt.plot(*interior.xy,color='b')
for g in router.skeleton.geoms:
    plt.plot(*g.xy,color='r')

In [9]:
skel=router.skeleton
skel.buffer(1)

def plot_pol(pol,color='k'):
    plt.plot(*pol.exterior.xy,color=color)
    for interior in pol.interiors:
        plt.plot(*interior.xy,color=color)
for i in range(10):
    plot_pol(skel.buffer(i))

In [10]:
def make_routes(gap,n_routes,skel,dev):
    all_lines=[]
    for n in range(n_routes):
        lines=skel.buffer((n+1)*gap).intersection(dev).boundary
        all_lines.append(lines)
    return sh.unary_union(sh.line_merge(all_lines))

In [11]:
route_lines=make_routes(gap=1.1,n_routes=5,skel=skel,dev=dev.buffer(-2))
all_lines = sh.line_merge(route_lines).union(skel)

In [12]:
for g in all_lines.geoms:
    plt.plot(*g.xy)

In [13]:
for g in skel.geoms:
    plt.plot(*g.xy)
for g in skel.boundary.geoms:
    plt.plot(*g.xy,'o')


In [14]:
waypoints=[]
for point in skel.boundary.geoms:
    waypoints.append(point.buffer(point.distance(dev.buffer(-2).boundary)))
criss_cross_lines = []
for g in sh.unary_union(waypoints).geoms:
    multiplicity=len(g.intersection(skel).geoms)
    if multiplicity==3:
        criss_cross_lines.append( sh.affinity.rotate(all_lines.intersection(g),angle=180,origin=g.centroid))
    else:
        criss_cross_lines.append( sh.affinity.rotate(all_lines.intersection(g),angle=45,origin=g.centroid))
criss_cross_lines=sh.unary_union(criss_cross_lines)

In [15]:
for g in dev.buffer(-2).boundary.geoms:
    plt.plot(*g.xy,color='k')
for g in all_lines.geoms:
    plt.plot(*g.xy,color='r')
for g in criss_cross_lines.geoms:
    plt.plot(*g.xy,color='b')

In [16]:
electrode_xy=[]
extra_lines=[]

for ang in np.arange(0,2*np.pi,2/3*np.pi):
    cx,cy=np.cos(ang)*35,np.sin(ang)*35
    for ang_on_ring  in [ang-np.pi/12,ang,ang+np.pi/12]:
        x,y=np.cos(ang_on_ring)*rad+cx,np.sin(ang_on_ring)*rad+cy
        x1,y1=np.cos(ang_on_ring)*(rad-5)+cx,np.sin(ang_on_ring)*(rad-5)+cy
        x2,y2=np.cos(ang_on_ring)*(rad+5)+cx,np.sin(ang_on_ring)*(rad+5)+cy
        electrode_xy.append([x,y])
        extra_lines.append((sh.LineString( [ sh.Point(x1,y1),sh.Point(x2,y2)] )))
new_all_lines = all_lines

electrode_footprints=[]
corrected_lines = []
for pos in electrode_xy:
    plt.plot(pos[0],pos[1],'o',color='k')
    electrode=sh.Point(pos).buffer(1.5)
    electrode_footprints.append(electrode)
    cut_points=electrode.boundary.intersection(all_lines)
    new_all_lines.difference(electrode)
    new_electrode_line = [sh.LineString([p,electrode.centroid]) for p in cut_points.geoms]
    corrected_lines.append(sh.unary_union(new_electrode_line))
#     for g in sh.unary_union(new_electrode_line).geoms:
#         plt.plot(*g.xy,color='b')

In [17]:
corrected_all_lines=new_all_lines.difference(sh.unary_union(electrode_footprints)).union(sh.unary_union(corrected_lines))

In [18]:
for g in corrected_all_lines.geoms:
    plt.plot(*g.xy)
for g in sh.unary_union(extra_lines).geoms:
    plt.plot(*g.xy)

In [19]:
electrode_extra_lines = sh.unary_union(extra_lines)

pad_lines = []
for i in range(4):
    ang=(90-20+i*10)*np.pi/180
    x1,y1 = np.cos(ang)*190,np.sin(ang)*190
    x2,y2 = np.cos(ang)*210,np.sin(ang)*210
    pad_lines.append(sh.LineString([sh.Point(x1,y1),sh.Point(x2,y2)]))
for i in range(5):
    ang=(270-20+i*10)*np.pi/180
    x1,y1 = np.cos(ang)*190,np.sin(ang)*190
    x2,y2 = np.cos(ang)*210,np.sin(ang)*210
    pad_lines.append(sh.LineString([sh.Point(x1,y1),sh.Point(x2,y2)]))


In [20]:
for g in sh.unary_union(pad_lines).geoms:
    plt.plot(*g.xy)

In [21]:
total_lines=corrected_all_lines.union(sh.unary_union(pad_lines)).union(electrode_extra_lines).union(criss_cross_lines)

In [231]:
for g in sh.unary_union(total_lines).geoms:
    plt.plot(*g.xy)

In [132]:
line_end_coords=[]
all_physical_lines=[]
for l in total_lines.geoms:
    bdy_coords=[list(el.coords)[0] for el in l.boundary.geoms]
    if len(bdy_coords)>0:
        line_end_coords.append(bdy_coords) #end point coords of lines
        all_physical_lines.append(l) #collect all lines into one list
#there are faulty lines in the merge, which fold back onto themselves (first point = last point), which technically does not have a boundary and is a closed polygon
#eliminate those
end_pts=np.concatenate(line_end_coords)
end_to_end_connections= np.asarray([(2*i,2*i+1) for i in range(len(end_pts)//2)]) #every consecutive 2 points are connected together by the LineString


from scipy.spatial import cKDTree
tree_elec=cKDTree(electrode_xy)
tree_pts=cKDTree(end_pts)          
elec_idx=[el[0] for el in tree_elec.query_ball_tree(tree_pts,r=1e-3)]
pad_idx = np.where( np.abs(np.sum(end_pts**2,axis=1)-210*210) < 1e-3)[0]


nodes_to_merge=tree_pts.query_ball_tree(tree_pts,r=1e-3)
for nodes in nodes_to_merge:
    if len(nodes)>0:
        for replaced in nodes[1:]: #replace all other neighbor nodes by the first node index, so intersections dont count as multiple
            end_to_end_connections[end_to_end_connections==replaced]=nodes[0]
            pad_idx[pad_idx==replaced]=nodes[0]



[2559, 2571, 2583, 2595, 2607, 2619, 2631, 2643, 2655]


In [135]:

import networkx as nx
Graph = nx.from_edgelist(end_to_end_connections)
#Graph.add_edges_from(edge_by_lines)
source=np.max(Graph.nodes)+1
sink=np.max(Graph.nodes)+2

for el in elec_idx:
    Graph.add_edge(el,source)
for el in pad_idx:
    Graph.add_edge(el,sink)
    

In [232]:
paths=list(nx.node_disjoint_paths(Graph,source,sink))
paths=[el[1:-1] for el in paths] #skip sink and source nodes
paths[0]

[2559,
 1473,
 831,
 829,
 827,
 443,
 441,
 337,
 373,
 371,
 369,
 367,
 365,
 363,
 361,
 359,
 315,
 313,
 411,
 1433,
 1325,
 2251,
 2143,
 1163,
 1099,
 1035,
 971,
 2857]

In [194]:
routes=[]
for p in paths:  
    line=[]
    color=np.random.rand(3)
    for i in range(len(p)-1):
        line_idx=np.where(np.sum(end_to_end_connections==p[i],axis=1) + np.sum(end_to_end_connections==p[i+1],axis=1) == 2)[0][0]
        x,y=np.asarray(all_physical_lines[line_idx].coords).T
        line.append(np.vstack((x,y)).T)
    
    routes.append(merge_line(line))

In [234]:
device=shapely_to_phidl(dev)
for pos in electrode_xy:
    device<<pg.circle(radius=1,layer=3).movex(pos[0]).movey(pos[1])
for r in routes:
    elec_route=shapely_to_phidl(sh.line_merge(r).buffer(.1))
    elec_route_sm=shapely_to_phidl(smooth_route(r).buffer(.1))
    #device<<elec_route.remap_layers({0:3})
    device<<elec_route_sm.remap_layers({0:5})
qp(device)

In [241]:
smr=[]
for r in routes:
    smr.append(smooth_route(r))
np.diff(np.asarray(smr[0].coords),axis=1)

array([[  36.46611731, -206.80962813],
       [  36.16374917, -205.09349217],
       [  35.89824817, -203.57851411],
       ...,
       [  85.28275897,  -11.83672668],
       [  84.40500479,  -12.43579716],
       [  83.29629131,  -12.94095226]])